In [1]:
import os
from crewai import Agent, Task, Crew, Process
from langchain_google_genai import ChatGoogleGenerativeAI
import sys

In [2]:
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
from src.tools import GitHubRepoTool

In [5]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.7,
    google_api_key=os.getenv("GEMINI_API_KEY")
)

In [10]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content="J'adore la programmation.", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--9a70e542-9ecf-4239-8338-eaf781d6a431-0', usage_metadata={'input_tokens': 20, 'output_tokens': 7, 'total_tokens': 27, 'input_token_details': {'cache_read': 0}})

In [17]:
# Create specialized agents
repo_analyzer = Agent(
    role='Repository Analyzer',
    goal='Analyze GitHub repositories to understand their structure, purpose, and technical details',
    backstory="""You are an expert software engineer who specializes in analyzing codebases. 
    You can quickly understand the purpose, architecture, and key features of any software project 
    by examining its code structure, dependencies, and configuration files.""",
    tools=[GitHubRepoTool()],
    llm=llm,
    verbose=True
)

readme_writer = Agent(
    role='Technical Documentation Writer',
    goal='Create comprehensive, well-structured README files that help users understand and use the project',
    backstory="""You are a technical writer with extensive experience in creating developer documentation. 
    You excel at translating complex technical information into clear, accessible documentation that 
    helps developers quickly understand how to install, configure, and use software projects.""",
    llm=llm,
    verbose=True
)

content_reviewer = Agent(
    role='Documentation Quality Reviewer',
    goal='Review and improve README content for clarity, completeness, and professional presentation',
    backstory="""You are a meticulous editor and documentation specialist who ensures that all technical 
    documentation meets high standards of clarity, accuracy, and professionalism. You have a keen eye 
    for detail and know what makes documentation truly helpful for developers.""",
    llm=llm,
    verbose=True)

In [18]:

def create_readme_generation_crew(github_url: str) -> str:
    """
    Create and execute a crew to generate a README for a GitHub repository
    """
    
    # Task 1: Analyze the repository
    analyze_task = Task(
        description=f"""
        Analyze the GitHub repository at {github_url} and provide a comprehensive analysis including:
        1. Project purpose and main functionality
        2. Programming language and key technologies used
        3. Project structure and important files
        4. Dependencies and requirements
        5. Key features based on code analysis
        6. Installation requirements
        7. Usage patterns from code examples
        
        Use the GitHub Repository Analyzer tool to gather this information.
        """,
        agent=repo_analyzer,
        expected_output="Detailed analysis of the repository structure, purpose, and technical details"
    )
    
    # Task 2: Write the README
    write_task = Task(
        description="""
        Based on the repository analysis, create a comprehensive README.md file that includes:
        
        1. **Project Title and Description**: Clear, engaging project name and description
        2. **Features**: Key functionality and capabilities
        3. **Installation**: Step-by-step installation instructions
        4. **Usage**: Code examples and usage instructions
        5. **API Documentation**: If applicable, document main functions/classes
        6. **Configuration**: Environment variables or config files
        7. **Contributing**: Guidelines for contributors
        8. **License**: License information
        9. **Contact/Support**: How to get help or contact maintainers
        
        Use proper Markdown formatting with headers, code blocks, lists, and links.
        Make it professional, clear, and helpful for developers.
        """,
        agent=readme_writer,
        expected_output="Complete README.md file with proper Markdown formatting",
        context=[analyze_task]
    )
    
    # Task 3: Review and improve
    review_task = Task(
        description="""
        Review the generated README and improve it by:
        1. Checking for clarity and readability
        2. Ensuring all sections are complete and helpful
        3. Verifying proper Markdown formatting
        4. Adding badges if appropriate
        5. Improving code examples and explanations
        6. Ensuring professional presentation
        7. Adding table of contents if needed
        
        Provide the final, polished README.md content.
        """,
        agent=content_reviewer,
        expected_output="Final, polished README.md file ready for use",
        context=[write_task]
    )
    
    # Create and run the crew
    crew = Crew(
        agents=[repo_analyzer, readme_writer, content_reviewer],
        tasks=[analyze_task, write_task, review_task],
        process=Process.sequential,
        verbose=True
    )
    
    result = crew.kickoff()
    return result


In [19]:

def main():
    """
    Main function to run the README generator
    """
    print("🚀 CrewAI README Generator")
    print("=" * 50)
    
    # Get GitHub repository URL from user
    github_url = "https://github.com/sarthak221995/Multi-Agent-Systems-with-CrewAI"
    
    if not github_url:
        print("❌ Please provide a valid GitHub repository URL")
        return
    
    if not github_url.startswith('https://github.com/'):
        print("❌ Please provide a valid GitHub URL (https://github.com/owner/repo)")
        return
    
    print(f"\n🔍 Analyzing repository: {github_url}")
    print("This may take a few minutes...\n")
    
    try:
        # Generate README
        readme_content = create_readme_generation_crew(github_url)
        
        # Save to file
        output_file = "README.md"
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(str(readme_content))
        
        print(f"\n✅ README generated successfully!")
        print(f"📝 Saved to: {output_file}")
        print("\nPreview:")
        print("-" * 50)
        print(str(readme_content)[:500] + "..." if len(str(readme_content)) > 500 else str(readme_content))
        
    except Exception as e:
        print(f"❌ Error generating README: {str(e)}")



In [23]:
if __name__ == "__main__":
    main()

🚀 CrewAI README Generator

🔍 Analyzing repository: https://github.com/sarthak221995/Multi-Agent-Systems-with-CrewAI
This may take a few minutes...












╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 97abcc4f-a478-485b-a0d9-ccee2fcd6fc2                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Provider List: https://docs.litellm.ai/docs/providers

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Repository Analyzer                                                                                     │
│                                                                                                                 │
│  Task:                                                                                                          │
│          Analyze the GitHub repository at https://github.com/sarthak221995/Multi-Agent-Systems-with-CrewAI and  │
│  provide a comprehensive analysis including:                                                                    │
│          1. Project purpose and main functionality                                                              │
│          2. Programming language and key technologies used                                                      │
│          3. Project structure and important files                                                               │
│          4. Dependencies and requirements                                                                       │
│          5. Key features based on code analysis                                                                 │
│          6. Installation requirements                                                                           │
│          7. Usage patterns from code examples                                                                   │
│                                                                                                                 │
│          Use the GitHub Repository Analyzer tool to gather this information.                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Provider List: https://docs.litellm.ai/docs/providers

Output()

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call.    │
│  You passed model=models/gemini-2.0-flash                                                                       │
│   Pass model as E.g. For 'Huggingface' inference endpoints pass in                                              │
│  `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


 An unknown error occurred. Please check the details below.



╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 3c7aa88c-95f7-41a6-b569-10acb4238b3a                                                                     │
│  Agent: Repository Analyzer                                                                                     │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 97abcc4f-a478-485b-a0d9-ccee2fcd6fc2                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [22]:
if __name__ == "__main__":
    main()

🚀 CrewAI README Generator

🔍 Analyzing repository: https://github.com/sarthak221995/Multi-Agent-Systems-with-CrewAI
This may take a few minutes...












╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 6a5ae3dc-b7d3-423f-8280-7298e7edab97                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Provider List: https://docs.litellm.ai/docs/providers

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Repository Analyzer                                                                                     │
│                                                                                                                 │
│  Task:                                                                                                          │
│          Analyze the GitHub repository at https://github.com/sarthak221995/Multi-Agent-Systems-with-CrewAI and  │
│  provide a comprehensive analysis including:                                                                    │
│          1. Project purpose and main functionality                                                              │
│          2. Programming language and key technologies used                                                      │
│          3. Project structure and important files                                                               │
│          4. Dependencies and requirements                                                                       │
│          5. Key features based on code analysis                                                                 │
│          6. Installation requirements                                                                           │
│          7. Usage patterns from code examples                                                                   │
│                                                                                                                 │
│          Use the GitHub Repository Analyzer tool to gather this information.                                    │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Provider List: https://docs.litellm.ai/docs/providers

Output()

╭─────────────────────────────────────────────────── LLM Error ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  ❌ LLM Call Failed                                                                                             │
│  Error: litellm.BadRequestError: LLM Provider NOT provided. Pass in the LLM provider you are trying to call.    │
│  You passed model=models/gemini-2.0-flash                                                                       │
│   Pass model as E.g. For 'Huggingface' inference endpoints pass in                                              │
│  `completion(model='huggingface/starcoder',..)` Learn more: https://docs.litellm.ai/docs/providers              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯


 An unknown error occurred. Please check the details below.



╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 5f3050e4-3e29-423d-9f59-db32b0a542a1                                                                     │
│  Agent: Repository Analyzer                                                                                     │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 6a5ae3dc-b7d3-423f-8280-7298e7edab97                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output:                                                                                                  │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯